## DPK 반납 정보를 가공하여 데이타프레임(df) 으로 만듬

In [1]:
from tool import *

In [43]:
df = pd.read_excel(get_filename())
df['DPK_ID'] = df['DPK_ID'].astype(str)
df['MS_DateTime'] = df['MS_DateTime'].astype(str)
df['MS_DateTime'] = pd.to_datetime(df['MS_DateTime'])
df

,DPK_ID,CUST_NAME,CUST_DPK_PN,QCI_DPK_PN,MS_DPK_PN,Status,UpdateDateTime,CUST_DateTime,MS_DateTime
0,3556120313835,LG,SAE30761501,AZDPKW11197,FQC-10428,RTN,20231212223642,20231026084319,2023-10-26 08:43:19
1,3556120313862,LG,SAE30761501,AZDPKW11197,FQC-10428,RTN,20231212220153,20231026084319,2023-10-26 08:43:19
2,3422115013190,LG,SAE30761301,AZDPKW11196,KW9-00609,RTN,20231212215731,20231102173300,2023-11-02 17:33:00
3,3422223665125,LG,SAE30762501,AZDPKW11161,KK3-00027,RTN,20231205142542,20231016093336,2023-10-16 09:33:36
4,3422115013192,LG,SAE30761301,AZDPKW11196,KW9-00609,RTN,20231212221441,20231102173300,2023-11-02 17:33:00
...,...,...,...,...,...,...,...,...,...
90,3556120311003,LG,SAE30761501,AZDPKW11197,FQC-10428,RTN,20231212224509,20231026084319,2023-10-26 08:43:19
91,3556120310954,LG,SAE30761501,AZDPKW11197,FQC-10428,RTN,20231212220442,20231026084319,2023-10-26 08:43:19
92,3422223665211,LG,SAE30762501,AZDPKW11161,KK3-00027,RTN,20231215184623,20231016093336,2023-10-16 09:33:36
93,3556120313856,LG,SAE30761501,AZDPKW11197,FQC-10428,RTN,20231212233932,20231026084319,2023-10-26 08:43:19


## 1. DPK 반납 요청 리스트의 데이타 이상 여부 확인

In [44]:
# DPK ID 반납 요청 개수 확인
row_qty = len(df['DPK_ID'])
return_qty = len(df['DPK_ID'].unique())
str_len_check = df['DPK_ID'].str.len()
print(f'1. DPK ID의 행의 개수: {row_qty}')
print(f'2. DPK ID의 반납 수량: {return_qty}')
print(f'3. DPK 자리수(13자리) 이상한 것의 개수 : {(str_len_check != 13).sum()}') # 요청 DPK의 자리수가 13자리로 되어 있는지 확인
print(f'4. DPK P/N 자리수(9자리) 이상한 것의 개수: {(df["MS_DPK_PN"].astype(str).str.len() != 9 ).sum()}') # DPK ID 의 자리수가 9자리로 되어 있는지 확인

1. DPK ID의 행의 개수: 95
2. DPK ID의 반납 수량: 95
3. DPK 자리수(13자리) 이상한 것의 개수 : 0
4. DPK P/N 자리수(9자리) 이상한 것의 개수: 0


In [45]:
df['DPK Name'] = df['MS_DPK_PN'].map(dpk_map) # DPK Name 컬럼에 DPK PN에 맞는 Name matching

In [46]:
print(df.groupby(['MS_DPK_PN', 'DPK Name']).size()) # DPK 종류별 수량 확인
print('총 수량 :', df.groupby(['MS_DPK_PN', 'DPK Name']).size().sum()) # DPK 종류별 수량 확인

MS_DPK_PN  DPK Name            
FQC-10428  Win 11 Pro              70
KK3-00027  Win 11 Home Plus        15
KUK-00003  Win 11 Home Advanced     1
KW9-00609  Win 11 Home              9
dtype: int64
총 수량 : 95


## PKID가 PC OA의 DPK Fullfillment DB에 기 포함되어 있는지 확인

In [47]:
with open('D:/Data/DPK Fullfillment DB.bin', 'rb') as f:
    ff_db = pickle.load(f)

In [48]:
ff_db.shape

(85602, 5)

In [49]:
dpk_dup = pd.DataFrame(columns=df.columns)
dpk_dup

,DPK_ID,CUST_NAME,CUST_DPK_PN,QCI_DPK_PN,MS_DPK_PN,Status,UpdateDateTime,CUST_DateTime,MS_DateTime,DPK Name


In [50]:
for row in df.iterrows():
    if row[1]['DPK_ID'] in ff_db['PKID'].tolist():
        dpk_dup = dpk_dup.append(row[1])
dpk_dup

,DPK_ID,CUST_NAME,CUST_DPK_PN,QCI_DPK_PN,MS_DPK_PN,Status,UpdateDateTime,CUST_DateTime,MS_DateTime,DPK Name


In [51]:
dpk_dup.shape

(0, 10)

## 1단계 : DPK return 전 반납하려는 DPK 의 Fullfillment 정보를 등록하기 위한 format 생성

In [52]:
# fullfillment 정보 등록하는 format의 컬럼명를 설정
col = ['Site', 'MS P/No.', 'PKID', 'FulfilledDateUTC', 'OEM PO Number', 'Date of Update', 'Remark']

In [53]:
f1 = datetime.date.today().strftime('%y%m%d')
return_PO_num = f'QT{f1}_01'

df1 = df[['MS_DPK_PN', 'DPK_ID', 'MS_DateTime' ]]
df1.insert(0, 'Site', 'QT')
df1['OEM PO Number'] = return_PO_num
df1['Date Of Update'] = None
df1['Remark'] = None
df1

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_10664\2853372322.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['OEM PO Number'] = return_PO_num
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_10664\2853372322.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Date Of Update'] = None
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_10664\2853372322.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

,Site,MS_DPK_PN,DPK_ID,MS_DateTime,OEM PO Number,Date Of Update,Remark
0,QT,FQC-10428,3556120313835,2023-10-26 08:43:19,QT231219_01,None,None
1,QT,FQC-10428,3556120313862,2023-10-26 08:43:19,QT231219_01,None,None
2,QT,KW9-00609,3422115013190,2023-11-02 17:33:00,QT231219_01,None,None
3,QT,KK3-00027,3422223665125,2023-10-16 09:33:36,QT231219_01,None,None
4,QT,KW9-00609,3422115013192,2023-11-02 17:33:00,QT231219_01,None,None
...,...,...,...,...,...,...,...
90,QT,FQC-10428,3556120311003,2023-10-26 08:43:19,QT231219_01,None,None
91,QT,FQC-10428,3556120310954,2023-10-26 08:43:19,QT231219_01,None,None
92,QT,KK3-00027,3422223665211,2023-10-16 09:33:36,QT231219_01,None,None
93,QT,FQC-10428,3556120313856,2023-10-26 08:43:19,QT231219_01,None,None


In [54]:
df1.columns = ['Site', 'MS P/No.', 'PKID', 'FulfilledDateUTC', 'OEM PO Number', 'Date of Update', 'Remark']

In [55]:
df1

,Site,MS P/No.,PKID,FulfilledDateUTC,OEM PO Number,Date of Update,Remark
0,QT,FQC-10428,3556120313835,2023-10-26 08:43:19,QT231219_01,None,None
1,QT,FQC-10428,3556120313862,2023-10-26 08:43:19,QT231219_01,None,None
2,QT,KW9-00609,3422115013190,2023-11-02 17:33:00,QT231219_01,None,None
3,QT,KK3-00027,3422223665125,2023-10-16 09:33:36,QT231219_01,None,None
4,QT,KW9-00609,3422115013192,2023-11-02 17:33:00,QT231219_01,None,None
...,...,...,...,...,...,...,...
90,QT,FQC-10428,3556120311003,2023-10-26 08:43:19,QT231219_01,None,None
91,QT,FQC-10428,3556120310954,2023-10-26 08:43:19,QT231219_01,None,None
92,QT,KK3-00027,3422223665211,2023-10-16 09:33:36,QT231219_01,None,None
93,QT,FQC-10428,3556120313856,2023-10-26 08:43:19,QT231219_01,None,None


In [56]:
df2 = df1[~df1['PKID'].isin(dpk_dup['DPK_ID'])] # 기 등록된 PKID는 제외함

In [57]:
ff_qty = return_qty - dpk_dup.shape[0] # fullfillment 내역으로 등록할 수량

In [58]:
df2.to_excel(f'D:/Python result/DPK Return/{f1}_Quanta_DPK_Fullfillment_register_{ff_qty}.xlsx', index = False)

## 2단계 : DPK 반납을 위한 DPK와 S/N의 매칭 정보(CBR)를 업로드
 * 현재 PCOA의 CBR 처리 방식은 PKID 가 이전에 등록이 되었는지 check 하여 중복된 PKID는 업로드시 에러를 발생시킴(SET S/N 은 중복여부를 check 하지 않음) - 23.4.17 확인(담당자 : 최재혁 책임)

In [59]:
df_cbr = pd.read_excel(get_filename(), sheet_name='95pcs PKID&SN mapping')
df_cbr['DPK_ID'] = df_cbr['DPK_ID'].astype(str)
df_cbr['QCI_SN'] = df_cbr['QCI_SN'].str.strip()

In [60]:
row_qty = len(df_cbr['DPK_ID'])
dpk_unique_qty = len(df_cbr['DPK_ID'].unique().tolist())
sn_unique_qty = len(df_cbr['QCI_SN'].unique().tolist())
str_len_check1 = df_cbr['DPK_ID'].str.len()
str_len_check2 = df_cbr['QCI_SN'].str.len()

print(f'1. DPK ID의 행의 개수: {row_qty}')
print(f'2. DPK ID의 수량: {dpk_unique_qty}')
print(f'3. SET S/N의 수량: {sn_unique_qty}')
print(f'4. DPK 자리수(13자리) 이상한 것의 개수 : {(str_len_check1 != 13).sum()}') # 요청 DPK의 자리수가 13자리로 되어 있는지 확인
print(f'5. Set S/N 자리수(13자리) 이상한 것의 개수 : {(str_len_check2 != 13).sum()}') # SET S/N의 자리수가 13자리로 되어 있는지 확인

1. DPK ID의 행의 개수: 95
2. DPK ID의 수량: 95
3. SET S/N의 수량: 93
4. DPK 자리수(13자리) 이상한 것의 개수 : 0
5. Set S/N 자리수(13자리) 이상한 것의 개수 : 0


In [61]:
df_cbr['QCI_SN'].value_counts()

311QCCV563939    2
311QCZP563941    2
312QCBD557317    1
311QCRN557118    1
312QCNL557335    1
                ..
312QCYQ556794    1
312QCRN557318    1
312QCCV556651    1
312QCQX557372    1
312QCGW557351    1
Name: QCI_SN, Length: 93, dtype: int64

In [63]:
df_cbr = df_cbr.drop_duplicates(subset=['DPK_ID'], keep='last') # 중복된 DPK ID를 삭제하는데 가장 마지막 행의 DPK ID만 남기고 삭제함

In [65]:
set(df['DPK_ID'].unique()) - set(df_cbr['DPK_ID'].unique()) # 2개의 데이타(반납할 DPK 리스트와 CBR 등록할 DPK 리스트)의 DPK 값이 완전히 일치하는 지 확인

set()

In [66]:
len(df_cbr['DPK_ID'].unique()) # 등록할 CBR 데이터의 unique 한 DPK ID 개수를 확인

95

In [67]:
with open('D:/Data/DPK CBR DB.bin', 'rb') as f:
    cbr_db = pickle.load(f)

df_cbr1 = df_cbr[~df_cbr['DPK_ID'].isin(cbr_db['PKID'])] # 기존에 CBR로 등록된 DPK ID 제외한 리스트로 만듬

In [68]:
df_cbr1 = df_cbr1[['DPK_ID', 'QCI_SN']].merge(df[['DPK_ID', 'MS_DPK_PN', 'MS_DateTime']])
df_cbr1['SITE'] = 'QT' # site 정보 추가
df_cbr1.columns = ['PKID', 'SET serial No.', 'MS P/No.', 'MSReceivedDateUTC ', 'SITE'] # 컬럼명 변경
df_cbr1['Mac Address'] = None # Mac address 컬럼 추가
df_cbr1['Remark'] = None # Remark 컬럼 추가
df_cbr1 = df_cbr1[['SITE', 'SET serial No.', 'MS P/No.', 'PKID', 'MSReceivedDateUTC ', 'Mac Address', 'Remark']] # 컬럼 순서를 PC OA의 CBR 등록 포맷에 맞게 변경

In [69]:
f1 = datetime.date.today().strftime('%y%m%d')
df_cbr1.to_excel(f'D:/Python result/DPK Return/{f1}_Quanta CBR register before DPK return_{df_cbr1.shape[0]}.xlsx', index = False)

## 3단계 : DPK RMA Return 포맷 작성

In [71]:
df_rma_return = df[['DPK_ID', 'MS_DPK_PN']].merge(df_cbr[['DPK_ID', 'QCI_SN']]) # Quanta가 요청하는 리턴 리스트(첫번째 시트)와 DPK와 S/N 맵핑 리스트를 merge하여 하나로 만듬
df_rma_return

,DPK_ID,MS_DPK_PN,QCI_SN
0,3556120313835,FQC-10428,312QCSF557374
1,3556120313862,FQC-10428,312QCZP557373
2,3422115013190,KW9-00609,312QCGW556775
3,3422223665125,KK3-00027,311QCSF563942
4,3422115013192,KW9-00609,312QCHE556793
...,...,...,...
90,3556120311003,FQC-10428,312QCTB557209
91,3556120310954,FQC-10428,312QCDG557282
92,3422223665211,KK3-00027,311QCZP563941
93,3556120313856,FQC-10428,312QCYQ557322


In [72]:
df_rma_return['Site'] = 'QT'
df_rma_return['Return Type'] = 'ZOC'
df_rma_return['No Credit'] = 'false'
df_rma_return[['ReturnDateUTC', 'Remark']] = None

df_rma_return.columns = ['PKID', 'MS P/No.', 'SET serial No.', 'Site', 'Return Type', 'No Credit', 'ReturnDateUTC', 'Remark']
df_rma_return = df_rma_return[['Site', 'SET serial No.', 'MS P/No.', 'PKID', 'Return Type', 'No Credit', 'ReturnDateUTC', 'Remark']]

In [73]:
df_rma_return

,Site,SET serial No.,MS P/No.,PKID,Return Type,No Credit,ReturnDateUTC,Remark
0,QT,312QCSF557374,FQC-10428,3556120313835,ZOC,false,None,None
1,QT,312QCZP557373,FQC-10428,3556120313862,ZOC,false,None,None
2,QT,312QCGW556775,KW9-00609,3422115013190,ZOC,false,None,None
3,QT,311QCSF563942,KK3-00027,3422223665125,ZOC,false,None,None
4,QT,312QCHE556793,KW9-00609,3422115013192,ZOC,false,None,None
...,...,...,...,...,...,...,...,...
90,QT,312QCTB557209,FQC-10428,3556120311003,ZOC,false,None,None
91,QT,312QCDG557282,FQC-10428,3556120310954,ZOC,false,None,None
92,QT,311QCZP563941,KK3-00027,3422223665211,ZOC,false,None,None
93,QT,312QCYQ557322,FQC-10428,3556120313856,ZOC,false,None,None


In [74]:
with open('D:/Data/DPK return db.bin', 'rb') as f: # DPK return DB 를 불러옴
    rt_db = pickle.load(f)

In [79]:
pkid_dup = df_rma_return[(df_rma_return['PKID'].isin(rt_db['PKID']))][['PKID', 'Remark']]  # DPK return DB에 등록했던 PKID 리스트(중복임)
pkid_dup['Remark'] = 'This DPK ID is already returned before by DPK return process'

In [80]:
pkid_dup

,PKID,Remark


In [81]:
df_rma_return = df_rma_return[~(df_rma_return['PKID'].isin(rt_db['PKID']))] # DPK return DB에 등록 이력이 없는 DPK만을 재구성

In [82]:
df_rma_return.shape

(95, 8)

In [83]:
with open('D:/Data/DPK RMA Return DB.bin', 'rb') as f: # DPK RMA return DB 를 불러옴
    rma_db = pickle.load(f)

In [84]:
pkid_dup = pd.concat([pkid_dup, df_rma_return[df_rma_return['PKID'].isin(rma_db['PKID'])][['PKID', 'Remark']]]).reset_index(drop=True) # DPK RMA return DB에 등록했던 PKID 리스트(중복임)
pkid_dup.loc[pkid_dup['Remark'].isnull(), 'Remark'] = 'This DPK ID is already returned before by DPK RMA return process'

In [85]:
df_rma_return = df_rma_return[~(df_rma_return['PKID'].isin(rma_db['PKID']))] # RMA 리스트 중 기존에 RMA return 이력이 있는 PKID는 제외하여 다시 구성
df_rma_return.shape

(95, 8)

In [86]:
sn_dup = df_rma_return[df_rma_return['SET serial No.'].isin(rma_db['SET Serial No.'])]

In [88]:
df_rma_return = df_rma_return[~df_rma_return['SET serial No.'].isin(rma_db['SET Serial No.'])] # Set S/N 중복 제외한 리스트

In [89]:
f1 = datetime.date.today().strftime('%y%m%d')
df_rma_return.to_excel(f'D:/Python result/DPK Return/{f1}_quanta_DPK RMA_return_{df_rma_return.shape[0]}.xlsx', index = False)

In [90]:
df_rma_return

,Site,SET serial No.,MS P/No.,PKID,Return Type,No Credit,ReturnDateUTC,Remark
0,QT,312QCSF557374,FQC-10428,3556120313835,ZOC,false,None,None
1,QT,312QCZP557373,FQC-10428,3556120313862,ZOC,false,None,None
2,QT,312QCGW556775,KW9-00609,3422115013190,ZOC,false,None,None
3,QT,311QCSF563942,KK3-00027,3422223665125,ZOC,false,None,None
4,QT,312QCHE556793,KW9-00609,3422115013192,ZOC,false,None,None
...,...,...,...,...,...,...,...,...
90,QT,312QCTB557209,FQC-10428,3556120311003,ZOC,false,None,None
91,QT,312QCDG557282,FQC-10428,3556120310954,ZOC,false,None,None
92,QT,311QCZP563941,KK3-00027,3422223665211,ZOC,false,None,None
93,QT,312QCYQ557322,FQC-10428,3556120313856,ZOC,false,None,None


In [73]:
sn_dup.to_clipboard()

In [145]:
pkid_dup.to_clipboard()

In [45]:
pkid_dup['PKID'][0]

'3258227938796'

In [42]:
sn_db = pd.read_clipboard()

In [35]:
df_rma_return[df_rma_return['PKID'].isin(rma_db['PKID'])]

,Site,SET serial No.,MS P/No.,PKID,Return Type,No Credit,ReturnDateUTC,Remark


In [45]:
df_rma_return[df_rma_return['SET serial No.'].isin(sn_db['SN'])]

,Site,SET serial No.,MS P/No.,PKID,Return Type,No Credit,ReturnDateUTC,Remark
29,QT,301QCUK559934,KW9-00609,3422106689326,ZOC,false,None,None
30,QT,305QCWC559009,KW9-00609,3422106689341,ZOC,false,None,None
63,QT,303QCAS559040,KK3-00027,3422217878532,ZOC,false,None,None
64,QT,111QCUK570270,KW9-00609,3422106689328,ZOC,false,None,None
65,QT,305QCZP558805,KW9-00609,3422106689339,ZOC,false,None,None
104,QT,301QCCV559467,KW9-00609,3422106689314,ZOC,false,None,None
105,QT,303QCMR556851,KK3-00027,3422217878533,ZOC,false,None,None
106,QT,305QCMR558859,KW9-00609,3422106689162,ZOC,false,None,None
